In [14]:
import pandas as pd
from datetime import datetime, date
from processing import Process

procdata = Process()

In [15]:
dftweet = pd.read_json('processed_data/twitter-results-2022-05-28_04-30-46.json')

dfmaster = pd.read_csv('processed_data/prouni-processado.csv')

In [16]:
dftweet.head(10)

,sentiment,positive,frequent,verified,date
0,-0.4215,False,"[[passei, 1], [terceira, 1], [chamada, 1], [uf...",False,2022-05-24 00:15:08+00:00
1,0.0000,False,"[[papel, 1], [fies, 1], [prouni, 1], [privatiz...",False,2022-05-23 23:36:59+00:00
2,0.0000,False,"[[políticas, 1], [fiscais, 1], [liberdade, 1],...",False,2022-05-24 00:20:46+00:00
3,0.0000,False,"[[vou, 1], [tentar, 1], [pro, 1], [prouni, 1],...",False,2022-05-23 22:39:58+00:00
4,0.4767,True,"[[esqueceu, 1], [abertura, 1], [contra, 1], [l...",False,2022-05-24 00:48:04+00:00
5,-0.6369,False,"[[3, 1], [chamada, 1], [passeikk, 1], [q, 1], ...",False,2022-05-23 22:34:32+00:00
6,0.6124,True,"[[escapei, 1], [tcc, 1], [porque, 1], [troquei...",False,2022-05-23 22:57:38+00:00
7,0.0000,False,"[[pro, 1], [prouni, 1], [recusaram, 1], [prime...",False,2022-05-23 23:59:09+00:00
8,0.5423,True,"[[ensino, 2], [educação, 1], [pt, 1], [recurso...",False,2022-05-23 23:21:52+00:00
9,0.0000,False,"[[prouni, 1], [mãos, 1]]",False,2022-05-23 19:17:58+00:00


In [17]:
print(dftweet.columns)
dftweet.count()
dftweet[dftweet['positive']==True]
dftweet[dftweet['verified']==True]
dftweet[dftweet['sentiment']!=0.0].count()


Index(['sentiment', 'positive', 'frequent', 'verified', 'date'], dtype='object')


sentiment    157
positive     157
frequent     157
verified     157
date         157
dtype: int64

In [18]:
tweets = dftweet['frequent'].agg('sum')
twtts = procdata.freq_tweet(tweets)

print("Sem tratamento", len(twtts))

tweetsfreq = pd.DataFrame(twtts, columns=['palavra', 'frequencia'])\
    .drop_duplicates(subset=['palavra'])\
    .sort_values(by=['frequencia'], ascending=False)

print("Com tratamento", tweetsfreq.__len__())
tweetsfreq.head(10)

Sem tratamento 1835
Com tratamento 951


,palavra,frequencia
8,prouni,150
50,q,45
119,pra,39
35,faculdade,27
108,ter,23
6,pq,21
152,enem,21
72,bolsa,19
54,2,19
211,100,17


In [50]:
tempdftweets = dftweet.copy()
tempdftweets['frequent'] = procdata.words_in_tweet(tempdftweets['frequent'])

tweetsfreq['sentimento'] = tweetsfreq['palavra'].apply(lambda w: procdata.get_sentiment_of_value(tempdftweets, w))
tweetsfreq = tweetsfreq.reset_index(drop=True)
tweetsfreq.head(10)

,palavra,frequencia,sentimento
0,prouni,150,Negativo
1,q,45,Negativo
2,pra,39,Negativo
3,faculdade,27,Negativo
4,ter,23,Positivo
5,pq,21,Negativo
6,enem,21,Positivo
7,bolsa,19,Negativo
8,2,19,Negativo
9,100,17,Negativo


In [49]:
sdates = dftweet['date'].map(lambda x: x.date()).drop_duplicates().reset_index(drop=True)
ssentdate = sdates.apply(lambda sd: dftweet.loc[dftweet['date'].apply(lambda d: d.date() == sd)]['sentiment'].mean())
temptweets = pd.DataFrame({
    'data': sdates,
    'sentimento': ssentdate
})
temptweets

,data,sentimento
0,2022-05-24,0.018400
1,2022-05-23,-0.105695
2,2022-05-22,-0.194256
3,2022-05-21,0.110800
4,2022-05-20,-0.100331
5,2022-05-19,-0.158281
6,2022-05-18,-0.026539
7,2022-05-17,-0.066903


In [108]:
stateslist = dfmaster.groupby(by='estado').size().sort_values(ascending=False).head(6).index
sstates = pd.Series(stateslist).reset_index(drop=True)

sstatesmensals = sstates.apply(lambda sd: dfmaster.loc[dfmaster['estado'].apply(lambda e: e == sd )]['mensalidade'].median())
bestmensals = pd.DataFrame({
    'estado': sstates,
    'mensalidade': sstatesmensals
})
bestmensals.sort_values(by='mensalidade', ascending=False)

,estado,mensalidade
4,BA,580.00
1,MG,536.04
0,SP,498.00
3,RS,443.00
2,PR,393.35
5,SC,393.35


In [ ]:
# TODO: Obter mensalidade de cada estado e se os estudantes terem tido acesso a internet durante a escola influencia

In [112]:
stateslist = dfmaster.groupby(by='estado').size().sort_values(ascending=False).head(6).index
sstates = pd.Series(stateslist).reset_index(drop=True)
sstatesmensals = sstates.apply(lambda state: dfmaster.loc[dfmaster['estado'].apply(lambda estado: estado == state)]['nota_bolsa'].median())
bestgrades = pd.DataFrame({
    'estado': sstates,
    'notabolsa': sstatesmensals.map(lambda x: round(x))
})
bestgrades.sort_values(by='notabolsa', ascending=False)

,estado,notabolsa
1,MG,583
4,BA,571
3,RS,567
0,SP,551
5,SC,550
2,PR,545


In [ ]:
# TODO: Obter nota das bolsas de cada estado e se a qualidade da escola influencia

In [26]:
dfmaster.groupby(by='estado').size().sort_values(ascending=False).head(6)

estado
SP    7277
MG    2586
PR    2244
RS    1891
BA    1676
SC    1139
dtype: int64